In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
df = pd.read_csv("../../scrapyproject/scrapyproject/file/naverPlace/여행랭크.csv")

In [4]:
df

,addr,category,context,id1,name,rank
0,강원도 홍천군 서면 팔봉리 1290-2,"숙박,콘도,리조트","스키장,스노우보드,실내워터파크,편안한,등산",13139708,소노벨 비발디파크,1
1,강원도 홍천군 서면 팔봉리 1290-2,"레저,테마,워터파크","실내워터파크,파도풀,노천온천,재미있는,휴식",11795570,오션월드,2
2,강원도 홍천군 두촌면 천현리 산134-1,"여행,명소,산,고개","휴양림,등산,막국수,편안한,계곡",11491290,가리산,3
3,강원도 홍천군 서면 팔봉리 1290-2,"숙박,콘도,리조트","스키장,실내수영장,산책로,편안한,휴식",13284318,소노펠리체 비발디파크,4
4,강원도 홍천군 두촌면 천현리 776,"자연명소,휴양림,산림욕장","휴양림,짚라인,캠핑,재미있는,등산",11490993,가리산자연휴양림,5
...,...,...,...,...,...,...
3208,경상북도 고령군 덕곡면 노리,"여행,명소,계곡","계곡,저수지,삼겹살,깨끗한,경치",21652251,상비리계곡,16
3209,경상북도 고령군 운수면 신간리 산123,"숙박,캠핑,야영장",NaN,1298840811,대가야 캠프타운,17
3210,경상북도 고령군 대가야읍 장기리 532,"여행,명소,보물","유적,암벽",20103855,고령 장기리 암각화,18
3211,경상북도 고령군 쌍림면 용리 187,"종교,불교","대나무,유물,저수지",11663973,반룡사,19


In [5]:
import time

In [6]:
driver = webdriver.Chrome('../chromedriver.exe')

In [7]:
xmf = pd.DataFrame(columns =['분위기', '인기토픽', '찾는목적', '인기월', '10', '20', '30', '40', '50', '60',
       'name'])
df2 = pd.DataFrame(columns= xmf.columns)
df2

,분위기,인기토픽,찾는목적,인기월,10,20,30,40,50,60,name


In [ ]:
start_time =time.time()
error = []
for nb , idx in enumerate(df['id1'].values[301:601]):
    md = {}
    tpc = {}
    obj = {}
    list1 = [md,tpc,obj]
    temp2 ={}
    tn = {}
    tw = {}
    th = {}
    fr = {}
    ft = {}
    sx= {}
    temp3 =[tn,tw,th,fr,ft,sx]
    try:
        driver.get('https://pcmap.place.naver.com/place/{}/home'.format(idx))
        time.sleep(1)
        spl =driver.find_element_by_link_text('더보기')
        spl.click()
        soup = BeautifulSoup(driver.page_source, 'html')
        div =  soup.select('div.place_detail_wrapper > div:nth-child(4) > div > div.place_section._1r18o')
        ul = div[0].select('div')[0].select('div ul')
        for j in range(3):
            for num, i in enumerate(ul[0].select('li')[j].select('span')):
                if num == 0:
                    tmp = i.text
                elif num ==1:
                    list1[j][tmp] = i.text.replace(', ','')
                else:
                     list1[j][tmp] +='|'+ i.text.replace(', ','')
        em = div[0].select('div')[0].select('h3 em')
        temp2['인기월'] = em[0].text
        ul2 = div[0].select('#bar_chart_container ul')
        for i in range(6):
            tmp = ul2[0].select('li')[i].text.split('%') 
            temp3[i][tmp[1][1:]] =tmp[0]
        xmf =pd.concat([pd.DataFrame(md,index= [nb]) ,pd.DataFrame(tpc, index=[nb]),pd.DataFrame(obj, index=[nb]),
                  pd.DataFrame(temp2, index=[nb]),pd.DataFrame(tn,index=[nb]),pd.DataFrame(tw, index=[nb]),
                  pd.DataFrame(th, index=[nb]),pd.DataFrame(fr, index=[nb]),pd.DataFrame(ft, index=[nb]),pd.DataFrame(sx, index=[nb])],axis=1)
        xmf['name'] = df.loc[nb,'name']

        df2 = pd.concat([df2,xmf],axis=0)
    
    except:
        error.append(nb)
    if nb % 300 == 0 and nb != 0 :
        end_time = time.time()
        print(start_time - end_time)
        start_time = time.time()

In [12]:
df2.to_excel('../selenium_file/naverPlace_300개.xlsx')

In [ ]:
dd
list1

In [88]:
temp2

{'인기월': '3월, 4월'}

In [133]:
df2 = pd.DataFrame(columns= xmf.columns)
df2 = pd.concat([df2,xmf],axis=0)
df2

,분위기,인기토픽,찾는목적,인기월,10,20,30,40,50,60,name
0,재미있는|깨끗한,계곡|캠핑|유원지|다슬기|참게탕,가족여행|힐링|휴식,"3월, 4월",25.00000000000001,65.66549999999997,89.49299999999995,86.05549999999997,25.00000000000001,25.00000000000001,오션월드


In [101]:
md.items(),tpc.items(),obj.items()

(dict_items([('분위기', '재미있는|깨끗한')]),
 dict_items([('인기토픽', '계곡|캠핑|유원지|다슬기|참게탕')]),
 dict_items([('찾는목적', '가족여행|힐링|휴식')]))

In [124]:
xmf =pd.concat([pd.DataFrame(md,index= [0]) ,pd.DataFrame(tpc, index=[0]),pd.DataFrame(obj, index=[0]),
          pd.DataFrame(temp2, index=[0]),pd.DataFrame(tn,index=[0]),pd.DataFrame(tw, index=[0]),
          pd.DataFrame(th, index=[0]),pd.DataFrame(fr, index=[0]),pd.DataFrame(ft, index=[0]),pd.DataFrame(sx, index=[0])],axis=1)
xmf['name'] = df.loc[1,'name']
xmf

,분위기,인기토픽,찾는목적,인기월,10,20,30,40,50,60,name
0,재미있는|깨끗한,계곡|캠핑|유원지|다슬기|참게탕,가족여행|힐링|휴식,"3월, 4월",25.00000000000001,65.66549999999997,89.49299999999995,86.05549999999997,25.00000000000001,25.00000000000001,오션월드
